In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import special

In [ ]:
df = pd.read_csv("../bin/run_2.out", delimiter=' ', names=('RA', 'Dec', 'z', 'L_gal', 'V_max', 'P_sat', 'M_halo', 'N_sat', 'L_tot', 'igrp', 'weight'))

In [ ]:
galprops = pd.read_csv("bin/sdss_galprops_v1.0.dat", delimiter=' ', names=('Mag_g', 'Mag_r', 'sigma_v', 'Dn4000', 'concentration', 'log_M_star'))
all_data = pd.merge(df, galprops, left_index=True, right_index=True)

In [ ]:
all_data

In [ ]:
bins = plt.hist(all_data['Mag_r'], bins=50)

In [ ]:
bins = plt.hist(np.log10(all_data['V_max']), bins=50)

In [ ]:
bins = plt.hist(np.log10(all_data['L_gal']), bins=50)

In [ ]:
def abs_mag_r_to_log_solar_L(a):
    """Converts an absolute magnitude to log solar luminosities using the sun's r-band magnitude."""
    return 0.4 * (4.65 - a)

# Test converstion of absolute mags (r band) to solar luminosities
plt.hist(abs_mag_r_to_log_solar_L(all_data['Mag_r']), bins=50)

In [ ]:
bins = np.logspace(np.log10(min(all_data.M_halo)), np.log10(max(all_data.M_halo)), 50)
labels = bins[0:len(bins)-1] # using bottom (or top?) value, not middle

all_data['Mh_bin'] = pd.cut(x = all_data['M_halo'], bins = bins, labels = labels, include_lowest = True)


In [ ]:
def D_crit(L_gal):
    return 1.42 + (0.35/2)*(1 + special.erf((np.log10(L_gal) - 9.9) / 0.8) )

x = np.logspace(np.log10(10**8), np.log10(10**13))
plt.plot(x, D_crit(x))
plt.xscale('log')

In [ ]:
all_data['D_crit'] = D_crit(all_data.L_gal)
all_data['starforming'] = all_data.Dn4000 < all_data.D_crit
all_data

In [ ]:
centrals = all_data[all_data.index == all_data.igrp]
sfc = centrals[centrals.starforming == True]
qc = centrals[centrals.starforming == False]
sfc


In [ ]:
means = centrals.groupby('Mh_bin').log_M_star.mean()
scatter = centrals.groupby('Mh_bin').log_M_star.std()

In [ ]:
plt.errorbar(labels, means, yerr=scatter)
plt.xscale('log')
plt.xlabel('$M_h / h$')
plt.ylabel('$\\log(M_\\star)$')


In [ ]:
plt.errorbar(labels, scatter)
plt.xscale('log')
plt.xlabel('$M_{h} / h$')
plt.ylabel('$\\sigma ( \\log M_\\star )$')

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter( centrals.M_halo, centrals.L_gal, alpha=0.002)
plt.loglog()
plt.xlabel('M_halo / h')
plt.ylabel('L_gal / $h^2$)')


In [ ]:
# Let's think about the color cut for r vs b galaxies

means_q = qc.groupby('Mh_bin').log_M_star.mean()
scatter_q = qc.groupby('Mh_bin').log_M_star.std()
means_sf = sfc.groupby('Mh_bin').log_M_star.mean()
scatter_sf = sfc.groupby('Mh_bin').log_M_star.std()

In [ ]:
plt.figure(figsize=(8,6))
plt.errorbar(labels, means_sf, yerr=scatter_sf, color='blue', alpha=1)
plt.errorbar(labels, means_q, yerr=scatter_q, color='red', alpha=1)
plt.xscale('log')
plt.xlabel('$M_h / h$')
plt.ylabel('$\\log(M_\\star)$')

In [ ]:
plt.errorbar(labels, scatter_sf, color='blue')
plt.errorbar(labels, scatter_q, color='red')
plt.xscale('log')
plt.xlabel('$M_{h} / h$')
plt.ylabel('$\\sigma ( \\log M_\\star )$')